In [1]:
import os
import zipfile
import glob
import pandas as pd
from PIL import Image
import random
import tensorflow as tf
from tensorflow import keras
from keras import layers
import numpy as np
from keras.applications import VGG16, MobileNet, ResNet50
from keras.layers import Dense, GlobalAveragePooling2D, BatchNormalization, Activation
from keras.models import Model
from keras.optimizers import Adam
from sklearn.model_selection import KFold
from keras.layers import Conv2D, MaxPooling2D, Flatten

# Dataset loading

In [2]:
# Set up Kaggle API Key
os.environ['KAGGLE_USERNAME'] = "xxxxxx"
os.environ['KAGGLE_KEY'] = "xxxxxx"

# Download the dataset
!kaggle datasets download -d samuelcortinhas/muffin-vs-chihuahua-image-classification

# Unzip the dataset
with zipfile.ZipFile("muffin-vs-chihuahua-image-classification.zip", "r") as zip_ref:
    zip_ref.extractall(".")

muffin-vs-chihuahua-image-classification.zip: Skipping, found more recently modified local copy (use --force to force download)


# Data preprocessing

### Reshape the images and convert to RGB

In [3]:
def load_images(image_dir, img_size):
    images = []
    for image_path in os.listdir(image_dir):
        img = Image.open(os.path.join(image_dir, image_path)).convert('RGB')  # Convert to RGB
        img_resized = img.resize((img_size, img_size))
        images.append(img_resized)
    return images


train_dir = 'train'
test_dir = 'test'
img_size = 224

# Load images
train_muffin_images = load_images(os.path.join(train_dir, 'muffin'), img_size)
train_chihuahua_images = load_images(os.path.join(train_dir, 'chihuahua'), img_size)
test_muffin_images = load_images(os.path.join(test_dir, 'muffin'), img_size)
test_chihuahua_images = load_images(os.path.join(test_dir, 'chihuahua'), img_size)


# Create DataFrames for train and test sets
train_data = {'image': train_muffin_images + train_chihuahua_images, 'label': ['muffin'] * len(train_muffin_images) + ['chihuahua'] * len(train_chihuahua_images)}
train_df = pd.DataFrame(train_data)

test_data = {'image': test_muffin_images + test_chihuahua_images, 'label': ['muffin'] * len(test_muffin_images) + ['chihuahua'] * len(test_chihuahua_images)}
test_df = pd.DataFrame(test_data)

# Print the DataFrames
print(train_df)
print(test_df)

                                                  image      label
0     <PIL.Image.Image image mode=RGB size=224x224 a...     muffin
1     <PIL.Image.Image image mode=RGB size=224x224 a...     muffin
2     <PIL.Image.Image image mode=RGB size=224x224 a...     muffin
3     <PIL.Image.Image image mode=RGB size=224x224 a...     muffin
4     <PIL.Image.Image image mode=RGB size=224x224 a...     muffin
...                                                 ...        ...
4728  <PIL.Image.Image image mode=RGB size=224x224 a...  chihuahua
4729  <PIL.Image.Image image mode=RGB size=224x224 a...  chihuahua
4730  <PIL.Image.Image image mode=RGB size=224x224 a...  chihuahua
4731  <PIL.Image.Image image mode=RGB size=224x224 a...  chihuahua
4732  <PIL.Image.Image image mode=RGB size=224x224 a...  chihuahua

[4733 rows x 2 columns]
                                                  image      label
0     <PIL.Image.Image image mode=RGB size=224x224 a...     muffin
1     <PIL.Image.Image image mode=RGB

### Convert the images into np arrays

In [4]:
def images_to_arrays(images):
    return np.stack([np.array(img) for img in images])

# vertical concatenatenation of arrays along the first axis
X_train = np.vstack((images_to_arrays(train_muffin_images), images_to_arrays(train_chihuahua_images)))
X_test = np.vstack((images_to_arrays(test_muffin_images), images_to_arrays(test_chihuahua_images)))

y_train = np.array([0] * len(train_muffin_images) + [1] * len(train_chihuahua_images))
y_test = np.array([0] * len(test_muffin_images) + [1] * len(test_chihuahua_images))

# Normalize the pixel values
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255

# Network architectures

## Training the model with 5-Fold CV

During the model.fit() both loss and accuracy for the training and for the validation set are computed, in order to monitoring the possible cases of overfitting. If the loss value decreases while the val_loss value starts to increase, it may indicate that the model is overfitting to the training set and is not generalizing well to new data.

### General model creation and training

In [5]:
def create_model(base_model, neurons, activation_function):
    for layer in base_model.layers: #transfer learning
        layer.trainable = False
    x = base_model.output
    x = Flatten()(x) #flatten the input tensor from a 3D tensor to a 2D tensor
    x = Dense(neurons)(x) 
    x = BatchNormalization()(x) #tries to make the process faster
    x = Activation(activation_function)(x) #introduces non-linearity
    predictions = Dense(1, activation='sigmoid')(x) #produces the binary classification output
    model = Model(inputs=base_model.input, outputs=predictions)
    return model

In [6]:
def train_and_evaluate_model(create_model, learning_rate, epochs, batch_size) : #these hyperparameters will be changed for experimenting
    # KFold cross-validation
    num_folds = 5
    kfold = KFold(n_splits=num_folds, shuffle=True, random_state=42)

    fold_scores = []
    fold_accuracies = []

    for train_index, val_index in kfold.split(X_train):
        X_train_fold, X_val = X_train[train_index], X_train[val_index]
        y_train_fold, y_val = y_train[train_index], y_train[val_index]

        # Create and compile the model
        model = create_model
        model.compile(optimizer=Adam(learning_rate=learning_rate), loss='binary_crossentropy', metrics=['accuracy'])

        # Train the model on the current fold
        history = model.fit(X_train_fold, y_train_fold, epochs=epochs, batch_size=batch_size, validation_data=(X_val, y_val))

        # Evaluate the model on the test set and store the performance metric (using zero-one loss as required)
        y_pred = model.predict(X_test)
        y_pred = np.round(y_pred)
        zero_one_loss = 1 - np.mean(y_pred.flatten() == y_test)
        fold_scores.append(zero_one_loss)
        fold_accuracies.append(np.mean(history.history['val_accuracy']))

    average_zero_one_loss = np.mean(fold_scores)
    average_accuracy = np.mean(fold_accuracies)
    print(f"Average zero-one loss across {num_folds} folds: {average_zero_one_loss}")
    print(f"Average accuracy across {num_folds} folds: {average_accuracy}")

    return average_zero_one_loss, average_accuracy

### from scratch CNN

In [78]:
def create_custom_cnn(input_shape):
    model = keras.Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    return model

In [79]:
base_CNN_model = create_custom_cnn(input_shape=(img_size, img_size, 3))

In [80]:
cnn_loss_1, cnn_accuracy_1 = train_and_evaluate_model(create_model(base_CNN_model, 1024, 'relu'), learning_rate=0.001, epochs=4, batch_size=8)

Epoch 1/4
474/474 [==============================] - 24s 49ms/step - loss: 0.6872 - accuracy: 0.5261 - val_loss: 0.6931 - val_accuracy: 0.5480
Epoch 2/4
474/474 [==============================] - 24s 52ms/step - loss: 0.6849 - accuracy: 0.5341 - val_loss: 0.6932 - val_accuracy: 0.4562
Epoch 3/4
474/474 [==============================] - 22s 46ms/step - loss: 0.6849 - accuracy: 0.5433 - val_loss: 0.6942 - val_accuracy: 0.5248
Epoch 4/4
37/37 [==============================] - 4s 114ms/step
Epoch 1/4
474/474 [==============================] - 27s 55ms/step - loss: 0.6851 - accuracy: 0.5378 - val_loss: 0.6943 - val_accuracy: 0.4984
Epoch 2/4
474/474 [==============================] - 23s 49ms/step - loss: 0.6836 - accuracy: 0.5446 - val_loss: 0.6863 - val_accuracy: 0.5312
Epoch 3/4
474/474 [==============================] - 23s 47ms/step - loss: 0.6838 - accuracy: 0.5486 - val_loss: 0.7041 - val_accuracy: 0.5312
Epoch 4/4
37/37 [==============================] - 4s 118ms/step
Epoch 1/4
47

In [81]:
cnn_loss_2, cnn_accuracy_2 = train_and_evaluate_model(create_model(base_CNN_model, 512, 'tanh'), learning_rate=0.0001, epochs=8, batch_size=16)

Epoch 1/8
237/237 [==============================] - 23s 93ms/step - loss: 0.6908 - accuracy: 0.5317 - val_loss: 0.6914 - val_accuracy: 0.5480
Epoch 2/8
237/237 [==============================] - 20s 84ms/step - loss: 0.6893 - accuracy: 0.5375 - val_loss: 0.6910 - val_accuracy: 0.5449
Epoch 3/8
237/237 [==============================] - 20s 84ms/step - loss: 0.6897 - accuracy: 0.5328 - val_loss: 0.6859 - val_accuracy: 0.5480
Epoch 4/8
237/237 [==============================] - 20s 85ms/step - loss: 0.6888 - accuracy: 0.5322 - val_loss: 0.6854 - val_accuracy: 0.5480
Epoch 5/8
237/237 [==============================] - 20s 85ms/step - loss: 0.6894 - accuracy: 0.5346 - val_loss: 0.6938 - val_accuracy: 0.5480
Epoch 6/8
237/237 [==============================] - 20s 85ms/step - loss: 0.6892 - accuracy: 0.5283 - val_loss: 0.6914 - val_accuracy: 0.5480
Epoch 7/8
237/237 [==============================] - 20s 85ms/step - loss: 0.6894 - accuracy: 0.5372 - val_loss: 0.6870 - val_accuracy: 0.5449

In [19]:
best_cnn_accuracy = max(cnn_accuracy_1, cnn_accuracy_2)
best_cnn_loss = min(cnn_loss_1, cnn_loss_2)

### VGG16

In [9]:
base_vgg_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_size, img_size, 3))

In [10]:
vgg_loss_1, vgg_accuracy_1 = train_and_evaluate_model(create_model(base_vgg_model, 1024, 'relu'), learning_rate=0.001, epochs=4, batch_size=8)

Epoch 1/4
474/474 [==============================] - 399s 841ms/step - loss: 0.2590 - accuracy: 0.9128 - val_loss: 0.0774 - val_accuracy: 0.9704
Epoch 2/4
474/474 [==============================] - 408s 861ms/step - loss: 0.0996 - accuracy: 0.9662 - val_loss: 0.0748 - val_accuracy: 0.9694
Epoch 3/4
474/474 [==============================] - 405s 854ms/step - loss: 0.0952 - accuracy: 0.9670 - val_loss: 0.0534 - val_accuracy: 0.9768
Epoch 4/4
37/37 [==============================] - 91s 2s/step
Epoch 1/4
474/474 [==============================] - 400s 842ms/step - loss: 0.1030 - accuracy: 0.9630 - val_loss: 0.0163 - val_accuracy: 0.9958
Epoch 2/4
474/474 [==============================] - 401s 847ms/step - loss: 0.0893 - accuracy: 0.9688 - val_loss: 0.0179 - val_accuracy: 0.9937
Epoch 3/4
474/474 [==============================] - 399s 841ms/step - loss: 0.0702 - accuracy: 0.9794 - val_loss: 0.0242 - val_accuracy: 0.9916
Epoch 4/4
37/37 [==============================] - 91s 2s/step
Epoc

In [11]:
vgg_loss_2, vgg_accuracy_2 = train_and_evaluate_model(create_model(base_vgg_model, 512, 'tanh'), learning_rate=0.0001, epochs=8, batch_size=16)

Epoch 1/8
237/237 [==============================] - 377s 2s/step - loss: 0.1845 - accuracy: 0.9411 - val_loss: 0.0556 - val_accuracy: 0.9799
Epoch 2/8
237/237 [==============================] - 373s 2s/step - loss: 0.0720 - accuracy: 0.9733 - val_loss: 0.0485 - val_accuracy: 0.9831
Epoch 3/8
237/237 [==============================] - 375s 2s/step - loss: 0.0287 - accuracy: 0.9894 - val_loss: 0.0463 - val_accuracy: 0.9842
Epoch 4/8
237/237 [==============================] - 372s 2s/step - loss: 0.0369 - accuracy: 0.9865 - val_loss: 0.0823 - val_accuracy: 0.9725
Epoch 5/8
237/237 [==============================] - 374s 2s/step - loss: 0.0219 - accuracy: 0.9923 - val_loss: 0.0454 - val_accuracy: 0.9884
Epoch 6/8
237/237 [==============================] - 374s 2s/step - loss: 0.0234 - accuracy: 0.9905 - val_loss: 0.0579 - val_accuracy: 0.9842
Epoch 7/8
237/237 [==============================] - 373s 2s/step - loss: 0.0131 - accuracy: 0.9966 - val_loss: 0.0742 - val_accuracy: 0.9768
Epoch 

In [18]:
best_vgg_accuracy = max(vgg_accuracy_1, vgg_accuracy_2)
best_vgg_loss = min(vgg_loss_1, vgg_loss_2)

### MobileNet

In [8]:
base_mobilenet_model = MobileNet(weights='imagenet', include_top=False, input_shape=(img_size, img_size, 3))

In [15]:
mobilenet_loss_1, mobilenet_accuracy_1 = train_and_evaluate_model(create_model(base_mobilenet_model, 1024, 'relu'), learning_rate=0.001, epochs=4, batch_size=8)

Epoch 1/4
474/474 [==============================] - 79s 161ms/step - loss: 0.1522 - accuracy: 0.9577 - val_loss: 0.0395 - val_accuracy: 0.9905
Epoch 2/4
474/474 [==============================] - 72s 152ms/step - loss: 0.0674 - accuracy: 0.9789 - val_loss: 0.0291 - val_accuracy: 0.9916
Epoch 3/4
474/474 [==============================] - 73s 154ms/step - loss: 0.0436 - accuracy: 0.9836 - val_loss: 0.0224 - val_accuracy: 0.9926
Epoch 4/4
37/37 [==============================] - 9s 221ms/step
Epoch 1/4
474/474 [==============================] - 76s 159ms/step - loss: 0.0555 - accuracy: 0.9855 - val_loss: 0.0023 - val_accuracy: 1.0000
Epoch 2/4
474/474 [==============================] - 74s 156ms/step - loss: 0.0328 - accuracy: 0.9892 - val_loss: 0.0037 - val_accuracy: 0.9989
Epoch 3/4
474/474 [==============================] - 74s 156ms/step - loss: 0.0595 - accuracy: 0.9844 - val_loss: 0.0058 - val_accuracy: 0.9979
Epoch 4/4
37/37 [==============================] - 8s 208ms/step
Epoch 

In [9]:
mobilenet_loss_2, mobilenet_accuracy_2 = train_and_evaluate_model(create_model(base_mobilenet_model, 512, 'tanh'), learning_rate=0.0001, epochs=8, batch_size=16)

Epoch 1/8
237/237 [==============================] - 40s 161ms/step - loss: 0.0912 - accuracy: 0.9765 - val_loss: 0.0522 - val_accuracy: 0.9884
Epoch 2/8
237/237 [==============================] - 38s 162ms/step - loss: 0.0541 - accuracy: 0.9839 - val_loss: 0.0334 - val_accuracy: 0.9947
Epoch 3/8
237/237 [==============================] - 39s 166ms/step - loss: 0.0164 - accuracy: 0.9947 - val_loss: 0.0287 - val_accuracy: 0.9926
Epoch 4/8
237/237 [==============================] - 39s 163ms/step - loss: 0.0189 - accuracy: 0.9950 - val_loss: 0.0394 - val_accuracy: 0.9905
Epoch 5/8
237/237 [==============================] - 38s 162ms/step - loss: 0.0081 - accuracy: 0.9963 - val_loss: 0.0303 - val_accuracy: 0.9947
Epoch 6/8
237/237 [==============================] - 39s 163ms/step - loss: 0.0056 - accuracy: 0.9987 - val_loss: 0.0306 - val_accuracy: 0.9937
Epoch 7/8
237/237 [==============================] - 38s 162ms/step - loss: 0.0082 - accuracy: 0.9960 - val_loss: 0.0395 - val_accuracy:

In [10]:
best_mobilenet_accuracy = max(mobilenet_accuracy_1, mobilenet_accuracy_2)
best_mobilenet_loss = min(mobilenet_loss_1, mobilenet_loss_2)

### ResNet50

In [11]:
base_resnet_model = ResNet50(weights='imagenet', include_top=False, input_shape=(img_size, img_size, 3))

In [12]:
resnet_loss_1, resnet_accuracy_1 = train_and_evaluate_model(create_model(base_resnet_model, 1024, 'relu'), learning_rate=0.001, epochs=4, batch_size=8)

Epoch 1/4
474/474 [==============================] - 224s 468ms/step - loss: 0.5312 - accuracy: 0.7750 - val_loss: 0.7045 - val_accuracy: 0.7075
Epoch 2/4
474/474 [==============================] - 224s 472ms/step - loss: 0.3873 - accuracy: 0.8310 - val_loss: 0.5319 - val_accuracy: 0.7614
Epoch 3/4
474/474 [==============================] - 223s 471ms/step - loss: 0.3319 - accuracy: 0.8616 - val_loss: 0.3886 - val_accuracy: 0.8237
Epoch 4/4
37/37 [==============================] - 34s 902ms/step
Epoch 1/4
474/474 [==============================] - 223s 466ms/step - loss: 0.3754 - accuracy: 0.8347 - val_loss: 0.2176 - val_accuracy: 0.9166
Epoch 2/4
474/474 [==============================] - 223s 472ms/step - loss: 0.3191 - accuracy: 0.8613 - val_loss: 1.8995 - val_accuracy: 0.5502
Epoch 3/4
474/474 [==============================] - 222s 469ms/step - loss: 0.2969 - accuracy: 0.8785 - val_loss: 0.8915 - val_accuracy: 0.7191
Epoch 4/4
37/37 [==============================] - 35s 919ms/ste

In [13]:
resnet_loss_2, resnet_accuracy_2 = train_and_evaluate_model(create_model(base_resnet_model, 512, 'tanh'), learning_rate=0.0001, epochs=8, batch_size=16)

Epoch 1/8
237/237 [==============================] - 174s 709ms/step - loss: 0.4678 - accuracy: 0.8069 - val_loss: 0.4544 - val_accuracy: 0.7825
Epoch 2/8
237/237 [==============================] - 157s 663ms/step - loss: 0.3172 - accuracy: 0.8719 - val_loss: 0.3394 - val_accuracy: 0.8606
Epoch 3/8
237/237 [==============================] - 159s 670ms/step - loss: 0.2844 - accuracy: 0.8822 - val_loss: 2.1618 - val_accuracy: 0.4879
Epoch 4/8
237/237 [==============================] - 160s 674ms/step - loss: 0.2294 - accuracy: 0.9023 - val_loss: 0.3721 - val_accuracy: 0.8310
Epoch 5/8
237/237 [==============================] - 160s 675ms/step - loss: 0.2180 - accuracy: 0.9142 - val_loss: 1.3759 - val_accuracy: 0.6103
Epoch 6/8
237/237 [==============================] - 161s 679ms/step - loss: 0.1821 - accuracy: 0.9287 - val_loss: 5.0487 - val_accuracy: 0.4794
Epoch 7/8
237/237 [==============================] - 160s 675ms/step - loss: 0.1644 - accuracy: 0.9382 - val_loss: 0.3476 - val_ac

In [14]:
best_resnet_accuracy = max(resnet_accuracy_1, resnet_accuracy_2)
best_resnet_loss = min(resnet_loss_1, resnet_loss_2)

## Results

In [32]:
data = {
    "Architecture": ["Custom CNN", "VGG16", "Mobilenet", "ResNet"] * 2,
    "Learning Rate": [0.001] * 4 + [0.0001] * 4,
    "Epochs": [4] * 4 + [8] * 4,
    "Batch Size": [8] * 4 + [16] * 4,
    "Activation Function": ["ReLU"] * 4 + ["tanh"] * 4,
    "Number of Neurons": [1024] * 4 + [512] * 4,
    "Accuracy": [cnn_accuracy_1, vgg_accuracy_1, mobilenet_accuracy_1, resnet_accuracy_1, cnn_accuracy_2, vgg_accuracy_2, mobilenet_accuracy_2, resnet_accuracy_2],
    "Loss": [cnn_loss_1, vgg_loss_1, mobilenet_loss_1, resnet_loss_1, cnn_loss_2, vgg_loss_2, mobilenet_loss_2, resnet_loss_2], 
}

df = pd.DataFrame(data)
print(df) 

  Architecture  Learning Rate  Epochs  Batch Size Activation Function  \
0   Custom CNN         0.0010       4           8                ReLU   
1        VGG16         0.0010       4           8                ReLU   
2    Mobilenet         0.0010       4           8                ReLU   
3       ResNet         0.0010       4           8                ReLU   
4   Custom CNN         0.0001       8          16                tanh   
5        VGG16         0.0001       8          16                tanh   
6    Mobilenet         0.0001       8          16                tanh   
7       ResNet         0.0001       8          16                tanh   

   Number of Neurons  Accuracy      Loss  
0               1024  0.529900  0.477872  
1               1024  0.991869  0.023649  
2               1024  0.998046  0.002365  
3               1024  0.817054  0.178378  
4                512  0.522950  0.486486  
5                512  0.994852  0.021115  
6                512  0.998284  0.002703 

In [20]:
print(f"Best custom CNN accuracy: {best_cnn_accuracy}\nBest VGG accuracy: {best_vgg_accuracy}\nBest MobileNet accuracy: {best_mobilenet_accuracy}\nBest ResNet accuracy: {best_resnet_accuracy}")

Best custom CNN accuracy: 0.5298998013138772
Best VGG accuracy: 0.9948520511388779
Best MobileNet accuracy: 0.9982840567827225
Best ResNet accuracy: 0.8170540183782578


In [21]:
print(f"Best custom CNN loss: {best_cnn_loss}\nBest VGG loss: {best_vgg_loss}\nBest MobileNet loss: {best_mobilenet_loss}\nBest ResNet loss: {best_resnet_loss}")

Best custom CNN loss: 0.47787162162162156
Best VGG loss: 0.021114864864864892
Best MobileNet loss: 0.0023648648648648906
Best ResNet loss: 0.1783783783783784


In [22]:
best_accuracy = max(best_vgg_accuracy, best_mobilenet_accuracy, best_resnet_accuracy)
best_loss = min(best_vgg_loss, best_mobilenet_loss, best_resnet_loss)

# Define the architecture names
architecture_names = ['VGG', 'MobileNet', 'ResNet']

# Initialize variables to store the best accuracy and loss
best_accuracy = 0.0
best_loss = float('inf')
best_accuracy_architecture = ''
best_loss_architecture = ''

# Iterate over the architectures
for i, architecture in enumerate([best_vgg_accuracy, best_mobilenet_accuracy, best_resnet_accuracy]):
    # Check if the current architecture has the best accuracy
    if architecture > best_accuracy:
        best_accuracy = architecture
        best_accuracy_architecture = architecture_names[i]
        
for i, architecture_loss in enumerate([best_vgg_loss, best_mobilenet_loss, best_resnet_loss]):
    # Check if the current architecture has the best loss
    if architecture_loss < best_loss:
        best_loss = architecture_loss
        best_loss_architecture = architecture_names[i]

# Print the architecture with the best accuracy and best loss
print(f"Best accuracy: {best_accuracy} from {best_accuracy_architecture}")
print(f"Best loss: {best_loss} from {best_loss_architecture}")

Best accuracy: 0.9982840567827225 from MobileNet
Best loss: 0.0023648648648648906 from MobileNet
